In [1]:
import torch.optim as optim
import Alpha0
from random import random, randrange
from battleships import Battleships
from battleships2 import Battleships2
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from copy import copy

from collections import deque
import MCTS
import progressbar as pb

game = Battleships()
game2 = Battleships2()

In [2]:
def placeShips(game):
        while True:
            boats = []
            directions = []

            for boat in game.ships:
                down: bool = random() >= 0.5
                directions.append(down)
                x: int = randrange(10 - (boat if not down else 0))
                y: int = randrange(10 - (boat if down else 0))
                boats.append((x, y))

            # If all 5 boat's positions are good we're done
            if game.place(boats, directions): break

In [4]:
policy = Alpha0.Policy()
policy = policy.cuda()
print(next(policy.parameters()).is_cuda)
optimizer = optim.Adam(policy.parameters(), lr=.01, weight_decay=1.e-5)

True


In [5]:
import MCTS
episodes = 600

widget = ['training loop: ', pb.Percentage(), ' ',
          pb.Bar(), ' ', pb.ETA()]
timer = pb.ProgressBar(widgets=widget, maxval=episodes).start()

outcomes = []
policy_loss = []

Nmax = 10

for e in range(episodes):
    #game = Battleships2()
    #game = Battleships()
    mytree = MCTS.Node(game)
    #print(game.state)
    #game.state = 0 * abs(game.state)
    #game.placeShips
    #game.placeShips
    placeShips(game)
    placeShips(game)

    logterm = []
    vterm = []

    while mytree.outcome is None:

        for _ in range(Nmax):
            mytree.explore(policy)
            if mytree.N >= Nmax:
                break
        #print(_)
        current_player = mytree.game.player
        mytree, (v, nn_v, p, nn_p) = mytree.next()
        mytree.detach_mother()

        loglist = torch.log(nn_p) * p
        
        constant = torch.where(p > 0, p * torch.log(p), torch.tensor(0.).cuda())
        logterm.append(-torch.sum(loglist - constant))
        

        vterm.append(nn_v * current_player)
        #print(game.state)
              
        #game.fire(mytree.game.last_move)
      

    # we compute the "policy_loss" for computing gradient
    outcome = mytree.outcome
    #print("10")
    outcomes.append(outcome)
    #print("12")
    #print(len(vterm))
    #print(torch.cuda.FloatTensor(vterm, requires_grad=True))
    test = torch.stack(vterm)
    #print("true")
    loss = torch.sum((torch.stack(vterm) - outcome) ** 2 + torch.stack(logterm))
    #print("13")
    optimizer.zero_grad()
    #print("14")
    loss.backward()
    #print("15")
    policy_loss.append(float(loss))
    #print("16")

    optimizer.step()
    #print("17")

    if e % 1 == 0:
        print("game: ", e + 1, ", mean loss: {:3.2f}".format(np.mean(policy_loss[-20:])),
              ", recent outcomes: ", outcomes[-10:])

    if e % 100 == 0:
        torch.save(policy, '6-6-4-pie-{:d}.mypolicy'.format(e))
    del loss
    #print(game.state)
    timer.update(e + 1)

timer.finish()

[[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

  [[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
   [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]]


 [[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [

ValueError: no children found and game hasn't ended

In [ ]:
mytree.explore(policy)

In [ ]:
input = input.to(device)
print(input)

In [ ]:
policy(input)

In [ ]:
mytree = MCTS.Node(game)
logterm = []
vterm = []
mytree.explore(policy)

current_player = mytree.game.player
current_player = mytree.game.player
mytree, (v, nn_v, p, nn_p) = mytree.next()
mytree.detach_mother()
loglist = torch.log(nn_p) * p
constant = torch.where(p > 0, p * torch.log(p), torch.tensor(0.).cuda())
logterm.append(-torch.sum(loglist - constant))

vterm.append(nn_v * current_player)
game.fire(mytree.game.last_move)

In [ ]:
print(nn_p)

In [ ]:
import matplotlib.pyplot as plt

% matplotlib notebook
plt.plot(losses)
plt.show()


In [ ]:
from battleships import Battleships as Game
from engines import InteractiveEngine, RandomEngine, A0, CompoundEngine

#   _____   _                       _                     _           _
#  / ____| | |                     | |                   (_)         | |
# | (___   | |__     ___     ___   | |_   _   _     ___   _   _ __   | | __  _   _      __ _    __ _   _ __ ___     ___
#  \___ \  | '_ \   / _ \   / _ \  | __| | | | |   / __| | | | '_ \  | |/ / | | | |    / _` |  / _` | | '_ ` _ \   / _ \
#  ____) | | | | | | (_) | | (_) | | |_  | |_| |   \__ \ | | | | | | |   <  | |_| |   | (_| | | (_| | | | | | | | |  __/
# |_____/  |_| |_|  \___/   \___/   \__|  \__, |   |___/ |_| |_| |_| |_|\_\  \__, |    \__, |  \__,_| |_| |_| |_|  \___|
#                                          __/ |                              __/ |     __/ |
#                                         |___/                              |___/     |___/

def play(player1Factory, player2Factory):
    """Play a game of Battleships
        player1Factory and player2Factory should be functions capable of turning
        a Battleships instance into an Engine for their respective players
    """
    game = Game()

    #Construct the two players from the given Engine factories
    p1 = player1Factory(game)
    p2 = player2Factory(game)

    #Get each engine to place down its ships
    p1.placeShips()
    p2.placeShips()

    while True:
        game.loud = p1.isTalkative()
        p1.attackShips()
        if game.score is not None: break  #Player 1 win

        game.loud = p2.isTalkative()
        p2.attackShips()
        if game.score is not None: break  #Player 2 win

    if not game.loud: print(f"Player {max(-game.player, 0) + 1} has won!")
    print(f"Game ended after {game.n_moves // 2} turns")

#if __name__ == "__main__":
    #play(InteractiveEngine, RandomEngine)

if __name__ == "__main__":
    play(A0, InteractiveEngine)
